In [1]:
import cv2
import numpy as np

image = cv2.imread('Images/bunchofshapes.jpg')
cv2.imshow('0 - Original Image', image)
cv2.waitKey(0)

image_blank = np.zeros((image.shape[0], image.shape[1], 3))

image_original = image.copy()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

edged = cv2.Canny(gray, 50, 200)
cv2.imshow('1 - Canny Edges', edged)
cv2.waitKey()

contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print("Number of contours found = ", len(contours))

cv2.drawContours(image_blank, contours, -1, (0,255,0),3)
cv2.imshow('2 - All contours over black image', image_blank)
cv2.waitKey()

cv2.drawContours(image, contours, -1, (0, 255, 0), 3)
cv2.imshow('3 - All contours', image)
cv2.waitKey()

cv2.destroyAllWindows()

Number of contours found =  4


In [2]:
# Sorting by Area.

def get_contour_areas(contours):
    # Returns the areas of all contours as list.
    all_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

print('Contour Areas before soring')
print(get_contour_areas(contours))

# Sort contours in descending order.
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

print('Contour areas after sorting')
print(get_contour_areas(sorted_contours))

# Iterate over contours and draw one at a time.
for c in sorted_contours:
    cv2.drawContours(image_original, [c], -1, (255, 0, 0), 3)
    cv2.waitKey()
    cv2.imshow('Contours by area', image_original)

cv2.waitKey()
cv2.destroyAllWindows()

Contour Areas before soring
[20587.5, 22901.5, 66579.5, 90222.0]
Contour areas after sorting
[90222.0, 66579.5, 22901.5, 20587.5]


In [3]:
# Sorting by position.

def x_cord_contour(contours):
    # Returns the X co-ordinate for the contour centroid.
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        print('M : \n', M )
        return (int(M['m10']/M['m00']))

def label_contour_center(image, c):
    # Places a red circle on the centers of contours.
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    
    cv2.circle(image, (cx,cy), 10,(0,0,255), -1)
    return image

for (i,c) in enumerate(contours):
    orig = label_contour_center(image,c)
    
cv2.imshow('4 - Contour centers ', image)
cv2.waitKey()

# Sort by left to right using our x_cord_contour function.
contours_left_to_right = sorted(contours, key = x_cord_contour, reverse = False)

# Labeling Contours Left to right.
for (i,c) in enumerate(contours_left_to_right):
    cv2.drawContours(image_original, [c], -1, (0,0,255), 3)
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cv2.putText(image_original, str(i+1), (cx,cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('6 - Left ro Right Contour', image_original)
    cv2.waitKey()
    (x,y,w,h) = cv2.boundingRect(c)
    
    # Crop each contour and save these images.
    cropped_contour = image_original[y:y+h,x:x+w]
    image_name = "OutputImages/output_shape_image_"+str(i+1)+".jpg"
    print(image_name)
    cv2.imwrite(image_name, cropped_contour)
    
cv2.destroyAllWindows()

M : 
 {'m00': 20587.5, 'm10': 18250924.833333332, 'm01': 9099782.333333332, 'm20': 16214606883.25, 'm11': 8067011518.125, 'm02': 4057711167.4166665, 'm30': 14436508422455.451, 'm21': 7166954589089.566, 'm12': 3597188928206.5664, 'm03': 1824964836980.2002, 'mu20': 35067975.07837677, 'mu11': 7590.238600730896, 'mu02': 35559934.190416336, 'mu30': -11432.6015625, 'mu21': 353952.1429042816, 'mu12': 359855.0387878418, 'mu03': -6583.537353515625, 'nu20': 0.08273770399700484, 'nu11': 1.7908046107890125e-05, 'nu02': 0.0838984088081491, 'nu30': -1.8799055832427544e-07, 'nu21': 5.820167929485615e-06, 'nu12': 5.9172314619470645e-06, 'nu03': -1.0825557560719243e-07}
M : 
 {'m00': 22901.5, 'm10': 13585477.833333332, 'm01': 10867630.833333332, 'm20': 8111037193.583333, 'm11': 6445296013.625, 'm02': 5206156291.916666, 'm30': 4873083197261.351, 'm21': 3848538181355.683, 'm12': 3086951262900.783, 'm03': 2515790732405.0503, 'mu20': 51949013.333509445, 'mu11': -1528770.2144794464, 'mu02': 49053048.4777059